In [95]:
%run crypt_backend.ipynb

# Parameter Generations
Generates the public parameters according to **Algorithm 3.2**.\
'parameter_generation' includes the rest of the functions, takes a given $q$ and a bit size for p.

In [97]:
#Find P functions
def isPrime(n):
    for i in range(2,int(n**0.5)+1):
        if n%i==0:
            return False 
    return True

def find_p_parameter(bit_size, q):
    lower = 2**(bit_size - 1)
    upper = 2**bit_size - 1
    primes = [p for p in range(lower, upper + 1) if isPrime(p)]
    rand.shuffle(primes)
    
    for candidate in primes:
        if candidate % q == 1:
            return candidate

    raise ValueError("No prime found in the range satisfying candidate % q == 1.")

In [98]:
def find_g(p,q):
    g=1
    while g==1:
        h=rand.randint(2,p-2)
        g=mod_exponentiate(h,(p-1)//q,p)
    return g

In [99]:
def parameter_generation(p_bit_size,q):
    p=find_p_parameter(p_bit_size,q)
    g=find_g(p,q)
    return p,q

# User Key Generation
Generates user public and private keys according to **Algorithm 3.3**.


In [101]:
def gen_user_keys(g,p,q):
    x_a=rand.randint(1,q-1)
    y_a=mod_exponentiate(g,x_a,p)

# Hash Function
Defines the toy hash function used in **Sections 3.3 and 3.4**.\
Converts a string to ASCII then sums the digits and returns the last 8 binary characters of this sum.

In [103]:
def toy_hash(string):
    hash=0
    for char in conv_to_ASCII(string):
       hash+=char
    if len(bin(hash))>=10:
        return bin(hash)[-8:]
    else:
        null_str='0'*(10-len(bin(hash)))
        return null_str+bin(hash)[2:]

# Sign To Blockchain
Takes as input public paramters and transaction information and a private key and outputs a signed block as in **Example 3.7**.\
Optional argument 'choose_k' allows for $k$ choice to be made manually for repeatability and testing, if empty, k is chosen randomly.\
If a block is the genesis block set 'prev_block=$0$'.

In [130]:
def sign_to_blockchain(p,q,g,sender,receiver,value,prev_block,private_key,choose_k=None):
    r=0
    prev_hash=''
    while r==0:
        if choose_k==None:
            k=rand.randint(1,q-1)
        else:
            k=choose_k
        r=mod_exponentiate(g,k,p)%q
        if r==0 and choose_k is not None:
            raise ValueError("Given k generates r=0")
    s=(modular_inverse(k,q)*((convert_binary_to_int(toy_hash(f'{sender}||{receiver}||{value}'))+(private_key*r))%q))%q
    if prev_block==0:
        prev_hash='0000'
    else:
        prev_hash=toy_hash(prev_block)
    block= f'{sender}||{receiver}||{value}||{r}||{s}||PrevHash={prev_hash}'
    return block

# Signature Verification
Verifies the signature on a given message by **Algorithm 3.5**.

In [107]:
def signature_verification(r,s,p,q,g,public_key,message):
    if r <= 0 or s<=0 or r>q or s>q:
        return 'Very Invalid'
    w=modular_inverse(s,q)
    v1=(convert_binary_to_int(toy_hash(message))*w)%q
    v2=(r*w)%q
    vinit=(mod_exponentiate(g,v1,p)*mod_exponentiate(public_key,v2,p))%p
    v=vinit%q
    if v==r:
        return 'Valid'
    elif v!=r:
        return 'Invalid'